In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt, Hannanum, Kkma
import re
import collections
from tqdm import tqdm

# 1. 데이터 불러오기

In [2]:
path = 'D:/Summer 2021/교류학술제/data_crawling/'
assembly_r = pd.read_excel(path + "국회.xlsx")
blue_r = pd.read_excel(path + '청와대(2).xlsx', ) # 결측치 채워진 버전

# 2. 불필요한 열 삭제

In [4]:
blue = blue_r.drop(['index', 'end', 'state'], axis = 1).copy()
assembly = assembly_r.drop(['state', 'end'], axis = 1).copy()

# 3. 불필요한 문자열 제거

In [5]:
blue.text = blue.text.str.rstrip('[본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]')

In [6]:
def remove_signs_from_text(df):
    for i in range(len(df)):
        df.iloc[i].text = re.sub('[^가-힝a-zA-Z\\s]', '', df.iloc[i].text)
    return df

In [7]:
assembly = remove_signs_from_text(assembly)
blue = remove_signs_from_text(blue)

In [8]:
def remove_comma_from_count(df):
    df['count'] = df['count'].str.replace(',', '')
    return df

In [9]:
assembly = remove_comma_from_count(assembly)
blue = remove_comma_from_count(blue)

# 4. 행 삭제

아래 토큰화 과정을 진행할 떄, 12386번째 행의 토큰화에서 지속적으로 문제가 발생함. 
확인해본 결과, 문자열에 문제가 있는 것으로 판단되어 불가피하게 12386번째 행을 삭제함. 

In [17]:
blue.iloc[12380:12390].text

12380    부동산 예비매수자들이 대거 몰려있는 국내 부동산 대형포털마다 신고가 얼마다 집값이 ...
12381    주 가맹본부의 갑질 행위를 사회에 고발하오니 엄격히 심판하여 주십시요    지난해 ...
12382    누구를 위한 국민재난지원금입니까 눈물로 호소합니다   존경하는 문재인 대통령님  저...
12383    자극적인 제목 죄송합니다 저 또한 그 사건을 떠올릴 때 마다 정신적으로 너무 고통스...
12384    학교폭력위원회에서 내려진 처벌규정이  호부터 호까지 있는데  호인 퇴학조치 이외의 ...
12385    저는 소방공무원 가족입니다  이번 코로나 백신 접종에 선택권을 준다지만 강제적이고 ...
12386                                                  ...
12387    저는 자칭 국민기업이라는 XX에서 X년 X월부터 XXXX XX으로 X년간 일하며 직...
12388    분양가를 시세의 로 한다니 이게 정상작인 나라인가요시세를 인위적으로 올리는곳도 많은...
12389    년 월 일 우리는 예견되어 있던 사고로 두 남매와 할머니 아들과 딸 장모님을 잃었습...
Name: text, dtype: object

In [19]:
blue = blue.drop([12386])
blue.iloc[12380:12390].text

12380    부동산 예비매수자들이 대거 몰려있는 국내 부동산 대형포털마다 신고가 얼마다 집값이 ...
12381    주 가맹본부의 갑질 행위를 사회에 고발하오니 엄격히 심판하여 주십시요    지난해 ...
12382    누구를 위한 국민재난지원금입니까 눈물로 호소합니다   존경하는 문재인 대통령님  저...
12383    자극적인 제목 죄송합니다 저 또한 그 사건을 떠올릴 때 마다 정신적으로 너무 고통스...
12384    학교폭력위원회에서 내려진 처벌규정이  호부터 호까지 있는데  호인 퇴학조치 이외의 ...
12385    저는 소방공무원 가족입니다  이번 코로나 백신 접종에 선택권을 준다지만 강제적이고 ...
12387    저는 자칭 국민기업이라는 XX에서 X년 X월부터 XXXX XX으로 X년간 일하며 직...
12388    분양가를 시세의 로 한다니 이게 정상작인 나라인가요시세를 인위적으로 올리는곳도 많은...
12389    년 월 일 우리는 예견되어 있던 사고로 두 남매와 할머니 아들과 딸 장모님을 잃었습...
12390    대통령님 보시기 바랍니다   코로나 로 사회가 부분 마비되고 경제가 어려운 가운데 ...
Name: text, dtype: object

# 5. 토큰화

In [10]:
def tokenize(df):
    text = df['text']
    kkm = Kkma()
    clean_words = pd.Series(0, index = df.index)
    for i in tqdm(range(len(text))):
        nouns = kkm.morphs(text.iloc[i])                 # 토큰화
        words = [w for w in nouns if len(w) >= 2]  # 분석을 위해 두글자 이상인 단어만 사용
        clean_words.iloc[i] = words
    clean_words = clean_words.astype(str)
    return clean_words

# 6. 불용어 제거

In [11]:
with open('D:/Summer 2021/교류학술제/data_cleaning/stopwords.txt', 'r', encoding = 'utf-8') as f:
    content = f.readlines()
stopwords = list(set([x.rstrip(', \n') for x in content]))

In [12]:
def remove_stopwords(series, stopwords):
    for word in stopwords:
        stopword = "'" + word + "', "
        series = series.str.replace(stopword, '')
    return series

# 국회 청원 (5. 토큰화 ~ 6. 불용어 제거 + 데이터 저장)

In [113]:
assembly.text = assembly_clean_words
assembly.to_excel("D:\Summer 2021\국회(전처리).xlsx", index = False)
assembly

,title,count,categ,start,text
0,국가보안법 개정에 관한 청원,1761,수사/법무/사법제도,2021-06-16,"['최근', '국가', '보안법', '폐지', '올라오', '자유', '대한민국',..."
1,인공지능 윤리 및 고인의 AI구현 법안 제정에 관한 청원,223,과학기술/정보통신,2021-06-15,"['동의', 'AI', '구현', '제재', '가하', '규정', '만들', '외의..."
2,포괄적 차별금지법 법안 제정에 관한 동의 및 일부 내용 수정 요청에 관한 청원,14025,인권/성평등/노동,2021-06-15,"['UN', '사회권', '위원회', '국가', '인권', '위원회', '포괄적',..."
3,국민연금 수급 개시 연령과 연계한 정년연장에 관한 청원,19807,인권/성평등/노동,2021-06-14,"['노동자', '정년', '기간', '국민', '연금', '수급', '개시', '연..."
4,하수도법 일부개정법률안에 관한 청원,736,재난/안전/환경,2021-06-11,"['하수도', '특정', '공산품', '주방', '오물', '분쇄', '기은', '..."
...,...,...,...,...,...
215,심리상담서비스법 제정에 관한 청원,226,복지/보훈,2021-07-07,"['최근', '한국', '심리', '학회', '상담', '심리', '학회', '임상..."
216,온라인 플랫폼 중개거래의 공정화에 관한 법률안 제정에 관한 청원,169,소비자/공정거래,2021-06-25,"['출판계', '플랫폼', '막아주', '한국', '출판계', '미래', '소설',..."
217,대체휴일에 관한 청원,161,행정/지방자치,2021-07-14,"['일요일', '공휴일', '대체', '휴일', '일괄적', '다음', '월요일',..."
218,셧다운제 폐지를 위한 청소년 보호법 개정에 관한 청원,152,저출산/고령화/아동/청소년/가족,2021-07-07,"['순기능', '제대로', '역효과', '일으키', '셧다', '운제', '청소년'..."


In [111]:
assembly_clean_words = remove_stopwords(assembly_words, stopwords)

In [24]:
assembly_words = tokenize(assembly)
assembly_words

# 청와대 청원 (5. 토큰화 ~ 6. 불용어 제거 + 데이터 저장)

청와대 청원의 경우, 데이터의 양이 너무 많아 토큰화에 시간이 오래 걸리고, 오류가 발생하면 데이터가 모두 날아가버리는 문제가 발생하기 때문에, 약 2000행 정도씩 묶어서 처리한 후 중간중간 저장할 수 있도록 했습니다. 

In [13]:
def tokenize_and_save(blue, j):
    if j + 2000 <= 15315:
        blue_words = tokenize(blue[j:j+2000])
    else:
        blue_words = tokenize(blue[j:])
    blue_clean_words = remove_stopwords(blue_words, stopwords)
    print('불용어 제거 완료')
    file_name = f"D:\Summer 2021\청와대(전처리)({j}).xlsx"
    blue_clean_words.to_excel(file_name, index = False)
    print('파일 저장 완료')

In [38]:
tokenize_and_save(blue, 0)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [35:09<00:00,  1.05s/it]


불용어 제거 완료
파일 저장 완료


In [48]:
tokenize_and_save(blue, 2000)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [34:16<00:00,  1.03s/it]


불용어 제거 완료
파일 저장 완료


In [50]:
tokenize_and_save(blue, 4000)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [46:04<00:00,  1.38s/it]


불용어 제거 완료
파일 저장 완료


In [52]:
tokenize_and_save(blue, 6000)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [43:13<00:00,  1.30s/it]


불용어 제거 완료
파일 저장 완료


In [54]:
tokenize_and_save(blue, 8000)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [51:43<00:00,  1.55s/it]


불용어 제거 완료
파일 저장 완료


In [55]:
tokenize_and_save(blue, 10000)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [46:06<00:00,  1.38s/it]


불용어 제거 완료
파일 저장 완료


In [20]:
tokenize_and_save(blue, 12000)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [59:12<00:00,  1.78s/it]


불용어 제거 완료
파일 저장 완료


In [22]:
tokenize_and_save(blue, 14000)

100%|██████████████████████████████████████████████████████████████████████████████| 1314/1314 [38:56<00:00,  1.78s/it]


불용어 제거 완료
파일 저장 완료


In [37]:
a = pd.read_excel("D:\Summer 2021\청와대(전처리)(0).xlsx")
b = pd.read_excel("D:\Summer 2021\청와대(전처리)(2000).xlsx")
c = pd.read_excel("D:\Summer 2021\청와대(전처리)(4000).xlsx")
d = pd.read_excel("D:\Summer 2021\청와대(전처리)(6000).xlsx")
e = pd.read_excel("D:\Summer 2021\청와대(전처리)(8000).xlsx")
f = pd.read_excel("D:\Summer 2021\청와대(전처리)(10000).xlsx")
g = pd.read_excel("D:\Summer 2021\청와대(전처리)(12000).xlsx")
h = pd.read_excel("D:\Summer 2021\청와대(전처리)(14000).xlsx")

In [38]:
blue_clean_words = pd.concat([a, b, c, d, e, f, g, h], ignore_index = True)
blue.text = blue_clean_words
blue.to_excel("D:\Summer 2021\교류학술제\data_cleaning\청와대(전처리).xlsx", index = False)
blue

,title,count,categ,start,text
0,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...,267,인권/성평등,2020-01-02,"['본인', '서울', '지방', '병무청', '검사장', '탈의실', '발견', ..."
1,주식시장 활성화 및 소액(개미)투자자 보호,271,경제민주화,2020-01-02,"['우리', '나라', '코스', '미국', '애플', '한국', '주식', '시장..."
2,교정기관의 민낮,198,행정,2020-01-02,"['억울', '국민', '신청', '구치소', '교도관', '이주', '구치소', ..."
3,미세먼지 저감 대책,170,안전/환경,2020-01-02,"['미세', '먼지', '심각', '이제', '적극적', '대안', '요구', '우..."
4,악질세입자 방지를 위한 세입자보호법을 재정해주세요.,2127,교통/건축/국토,2020-01-02,"['우선', '아이', '부모', '식구', '아이', '성장', '나름', '단꿈..."
...,...,...,...,...,...
15310,"충북 오창 사는 33살 동생이 신경외과에서 주사 한대로 심정지, 사망하였습니다",16307,보건복지,2021-07-05,"['안녕', '고등학교', '재학', '중인', '고등학생', '여러', '매체',..."
15311,오직 코로나 백신을 위해 9평 응시 신청을 한 분들의 응시 권한을 재수생과 검정고시...,387,보건복지,2021-07-05,"['대한민국', '대의', '사회', '초년', '이자', '취업', '준비', '..."
15312,보이스피싱 피해자의 억울함을 호소합니다. 진화하고 있는 신종 보이스피싱에 대한 수사...,352,안전/환경,2021-07-05,"['안녕', '사기죄', '무엇', '저희', '엄마', '께서', 'LH', '세..."
15313,사기죄? 왜 아닙니까?,213,기타,2021-07-05,"['대통령', '간곡히', '제언', '드리', '코로나', '이상', '휩쓸', ..."
